In [1]:
import torch
from torch import nn
import torchvision
import onnx
from onnx import numpy_helper
from collections import OrderedDict

/home/noah/anaconda3/envs/rps-pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class RpsModel(nn.Module):
    def __init__(self):
        super(RpsModel, self).__init__()
        # self.backbone = torchvision.models.efficientnet_b0()
        self.backbone = torchvision.models.mobilenet_v3_small()
        # self.backbone = mobile_net_v3.MobileNetV3(input_size=(3, 256, 256), except_head=True)
        self.fcn = nn.Linear(1000, 3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x / 127.5 - 1
        x = self.backbone(x)
        x = self.fcn(x)
        x = self.sigmoid(x)
        return x

In [3]:
model = RpsModel()
ckpt = torch.load('model/rps1a.pth')
model.load_state_dict(ckpt.state_dict())

<All keys matched successfully>

In [9]:
model.to('cuda')
dummy_input = torch.randn(1, 3, 256, 256, device="cuda")
input_name = ["input_1"]
output_name = ["output_1"]

torch.onnx.export(model, dummy_input, "model/rps1_cuda.onnx", verbose=True, input_names=input_name, output_names=output_name)

graph(%input_1 : Float(1, 3, 256, 256, strides=[196608, 65536, 256, 1], requires_grad=0, device=cuda:0),
      %backbone.features.1.block.1.fc1.weight : Float(8, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=1, device=cuda:0),
      %backbone.features.1.block.1.fc1.bias : Float(8, strides=[1], requires_grad=1, device=cuda:0),
      %backbone.features.1.block.1.fc2.weight : Float(16, 8, 1, 1, strides=[8, 1, 1, 1], requires_grad=1, device=cuda:0),
      %backbone.features.1.block.1.fc2.bias : Float(16, strides=[1], requires_grad=1, device=cuda:0),
      %backbone.features.4.block.2.fc1.weight : Float(24, 96, 1, 1, strides=[96, 1, 1, 1], requires_grad=1, device=cuda:0),
      %backbone.features.4.block.2.fc1.bias : Float(24, strides=[1], requires_grad=1, device=cuda:0),
      %backbone.features.4.block.2.fc2.weight : Float(96, 24, 1, 1, strides=[24, 1, 1, 1], requires_grad=1, device=cuda:0),
      %backbone.features.4.block.2.fc2.bias : Float(96, strides=[1], requires_grad=1, device=cuda